#### Coversion delta to csv for loading in Snowfalke datawarehouse

In [0]:
# service principal for connecting with ADLS and access it's data

spark.conf.set("fs.azure.account.auth.type.hpadlsacc.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.hpadlsacc.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "app-key"))
spark.conf.set("fs.azure.account.oauth2.client.secret.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "service-cred"))
tenant_id = dbutils.secrets.get("hc-secret-scope", "dir-id")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.hpadlsacc.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
# converting the delta tables to csv, for pushing it into the Snowflake database

# creating a list of gold table names  
tables = ["dim_claims", "dim_patient", "dim_provider", "dim_department" ,"fact_transactions", "dim_encounter"]

# passing it to the for loop, reading from the gold container and loading it to the goldcsv container in Azure Data Lake
for table in tables:

    df = spark.read.format("delta").load(f"abfss://gold@hpadlsacc.dfs.core.windows.net/{table}")

    df.write \
      .format("csv") \
      .option("header", "true") \
      .mode("overwrite") \
      .save(f"abfss://goldcsv@hpadlsacc.dfs.core.windows.net/{table}")
      
